# cuslide2 Plugin with nvImageCodec GPU Acceleration

This notebook demonstrates the new **cuslide2** plugin that provides GPU-accelerated JPEG/JPEG2000 decoding for digital pathology images using NVIDIA's nvImageCodec library.

## Features
- 🚀 **GPU-accelerated decoding** for JPEG and JPEG2000 compressed tiles
- 📊 **Performance benchmarking** comparing CPU vs GPU decode times
- 🔧 **Automatic plugin configuration** with priority handling
- 📁 **Support for multiple formats**: Aperio SVS, Philips TIFF, Generic TIFF
- ⚡ **Seamless fallback** to CPU decoders when GPU is unavailable


## Prerequisites

Ensure you have the following installed:
- cuCIM with cuslide2 plugin built
- nvImageCodec library (for GPU acceleration)
- CUDA-capable GPU (optional, will fallback to CPU)


In [ ]:
# Install prerequisites if needed
# !pip install cucim numpy pillow matplotlib
# !micromamba install libnvimgcodec-dev libnvimgcodec0 -c conda-forge  # For GPU acceleration


## Setup cuslide2 Plugin Configuration


In [3]:
import os
import sys
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def setup_cuslide2_plugin():
    """Configure cuCIM to use cuslide2 plugin with priority"""
    
    print("🔧 Setting up cuslide2 plugin...")
    
    # Set plugin root to build directory
    plugin_root = "/home/cdinea/cucim/build-release/lib"
    
    try:
        from cucim.clara import _set_plugin_root
        _set_plugin_root(plugin_root)
        print(f"✅ Plugin root set: {plugin_root}")
    except ImportError:
        print("❌ cuCIM not available - please install cuCIM")
        return False
    
    # Create plugin configuration to prioritize cuslide2
    config = {
        "plugin": {
            "names": [
                "cucim.kit.cuslide2@25.10.00.so",  # cuslide2 with nvImageCodec (highest priority)
                "cucim.kit.cuslide@25.10.00.so",   # Original cuslide (fallback)
                "cucim.kit.cumed@25.10.00.so"      # Medical imaging
            ]
        }
    }
    
    # Write config file
    config_path = "/tmp/.cucim_cuslide2_notebook.json"
    with open(config_path, "w") as f:
        json.dump(config, f, indent=2)
    
    # Set environment variable
    os.environ["CUCIM_CONFIG_PATH"] = config_path
    print(f"✅ Plugin configuration created: {config_path}")
    
    return True

# Setup the plugin
setup_success = setup_cuslide2_plugin()
if not setup_success:
    raise RuntimeError("Failed to setup cuslide2 plugin")


🔧 Setting up cuslide2 plugin...
✅ Plugin root set: /home/cdinea/cucim/build-release/lib
✅ Plugin configuration created: /tmp/.cucim_cuslide2_notebook.json
